In [1]:
from sklearn.model_selection import KFold
import pandas as pd

In [2]:
import wandb
wandb.login()

wandb: Currently logged in as: konggedzu (imucs). Use `wandb login --relogin` to force relogin


True

In [3]:
sweep_config = {
    'method': 'random'
}

In [4]:
metric = {
    'name': 'Grand Mean',
    'goal': 'maximize'
}

sweep_config['metric'] = metric

In [5]:
parameters_dict = {
    'dropout': {
          'values': [0.05, 0.1,0.3, 0.4, 0.5, 0.6, 0.7]
        },

    'batch_size': {
        'values': [4096]
        },

    'data_augmentation_multiple': {
        'values': [3, 5]
        }
}

In [6]:
sweep_config['parameters'] = parameters_dict

In [7]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-sub_loc_no_auto_threshold")

Create sweep with ID: ar9df2oq
Sweep URL: https://wandb.ai/imucs/pytorch-sweeps-sub_loc_no_auto_threshold/sweeps/ar9df2oq


In [8]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'Grand Mean'},
 'parameters': {'batch_size': {'values': [4096]},
                'data_augmentation_multiple': {'values': [3, 5]},
                'dropout': {'values': [0.05, 0.1, 0.3, 0.4, 0.5, 0.6, 0.7]}}}


In [9]:
feature_pd = pd.read_csv('/home/kongge/projects/new_protT5/data/DPC_T5_578_right.csv')
labels_pd = pd.read_csv("/home/kongge/projects/new_protT5/data/mutil_label_578.csv")

In [10]:
from dataAug.tools import MLDA
from dataAug.all_tools import dataAugSMOTE

In [11]:
smote_multiple = {}

In [12]:
import time
from classify.targeTools import testThresholdFive, Accuracy, countScore
from torch import optim
from torch.utils.data import DataLoader, TensorDataset
import torch
from Classify_adjust import ModelClassify
def train_and_val(config=None):
    with wandb.init(config=config):
        config = wandb.config
        kf = KFold(n_splits=10, shuffle=True)
        model_discord = []
        for train_index, test_index in kf.split(feature_pd):
            train_true_data = feature_pd.iloc[train_index]
            train_true_label = labels_pd.iloc[train_index]

            multi_label_samples = train_true_label[(train_true_label.sum(axis=1) >= 2)]
            multi_label_indices = multi_label_samples.index
            multi_features_samples = train_true_data.loc[multi_label_indices]
            ML_G_X, ML_G_y = MLDA(multi_features_samples, multi_label_samples, config.data_augmentation_multiple)

            G_feature, G_label = dataAugSMOTE(train_true_data, train_true_label, config.data_augmentation_multiple, 1424)
            G_feature = pd.concat([G_feature, ML_G_X], axis=0)
            G_label = pd.concat([G_label, ML_G_y], axis=0)
            train_feature = pd.concat([train_true_data, G_feature], axis=0)
            train_label = pd.concat([train_true_label, G_label], axis=0)

            test_data = feature_pd.iloc[test_index]
            test_label = labels_pd.iloc[test_index]

            datasetTrain = TensorDataset(torch.tensor(train_feature.values), torch.tensor(train_label.values))
            batch_size = config.batch_size
            dataloaderTrain = DataLoader(datasetTrain, batch_size=batch_size, shuffle=True)

            datasetTest = TensorDataset(torch.tensor(test_data.values), torch.tensor(test_label.values))

            batch_size = len(datasetTest)
            dataloaderTest = DataLoader(datasetTest, batch_size=batch_size, shuffle=False)

            model = ModelClassify(drop_rate=config.dropout, num_class=5, feature_num=1424)
            criterion = torch.nn.BCELoss()
            optimizer = optim.Adam(model.parameters(), lr=0.001)
            import warnings
            warnings.filterwarnings("ignore")
            GMList = {}
            epochs = 300
            for epoch in range(epochs):
                model.train()
                total_loss = 0.0
                for idx, data in enumerate(dataloaderTrain, 0):
                    inputs, labels = data
                    labels = labels.float()
                    inputs = inputs.float()
                    out = model(inputs)
                    loss = criterion(out, labels)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    total_loss += loss.item()
                avg_loss = total_loss / len(dataloaderTrain)
                threshold = 0.5
                labels_cov = torch.where(out > threshold, torch.tensor(1), torch.tensor(0))
                print(f"Epoch [{epoch+1}/{epochs}], Average Loss: {avg_loss:.4f}, ACC: {Accuracy(labels.int(), labels_cov)}")
                model.eval()
                with torch.no_grad():
                    for idxTest, dataTest in enumerate(dataloaderTest, 0):
                        inputsTest, targetsFive = dataTest
                        inputsTest = inputsTest.float()
                        test_output = model(inputsTest)
                test_labels_cov = torch.where(test_output > threshold, torch.tensor(1), torch.tensor(0))
                GMScore1 = countScore(targetsFive.int(), test_labels_cov)
                GMList[epoch] = GMScore1
            t = time.time()
            best_key = max(GMList, key=GMList.get)
            best_value = GMList[best_key]
            best_value = [x.item() if isinstance(x, torch.Tensor) else x for x in best_value]
            model_discord.append(best_value)
        model_discord_column_means = [sum(col) / len(col) for col in zip(*model_discord)]
        smote_multiple[int(t)] = model_discord_column_means
        wandb.log({"Grand Mean": model_discord_column_means[0]})

In [13]:
wandb.agent(sweep_id, train_and_val, count=20)

wandb: Agent Starting Run: qpgc9h4z with config:
wandb: 	batch_size: 4096
wandb: 	data_augmentation_multiple: 3
wandb: 	dropout: 0.6


/home/kongge/.conda/envs/kongge_pytorch_cpu/lib/python3.7/site-packages/imblearn/utils/_validation.py:314: UserWarning: After over-sampling, the number of samples (716) in class 0 will be larger than the number of samples in the majority class (class #1 -> 341)
  f"After over-sampling, the number of samples ({n_samples})"
/home/kongge/.conda/envs/kongge_pytorch_cpu/lib/python3.7/site-packages/imblearn/utils/_validation.py:314: UserWarning: After over-sampling, the number of samples (836) in class 0 will be larger than the number of samples in the majority class (class #1 -> 311)
  f"After over-sampling, the number of samples ({n_samples})"


Epoch [1/300], Average Loss: 0.7426, ACC: 0.1935126101488897
Epoch [2/300], Average Loss: 0.7041, ACC: 0.24484958979033597
Epoch [3/300], Average Loss: 0.6731, ACC: 0.29998480704952923
Epoch [4/300], Average Loss: 0.6441, ACC: 0.341742631419024
Epoch [5/300], Average Loss: 0.6213, ACC: 0.3769522941355235
Epoch [6/300], Average Loss: 0.5932, ACC: 0.4014205408690402
Epoch [7/300], Average Loss: 0.5723, ACC: 0.42065481616530326
Epoch [8/300], Average Loss: 0.5469, ACC: 0.4534032209055039
Epoch [9/300], Average Loss: 0.5270, ACC: 0.47349589790337665
Epoch [10/300], Average Loss: 0.5056, ACC: 0.4952825888787632
Epoch [11/300], Average Loss: 0.4861, ACC: 0.5185126101488922
Epoch [12/300], Average Loss: 0.4673, ACC: 0.5382102704345186
Epoch [13/300], Average Loss: 0.4531, ACC: 0.5473640230932856
Epoch [14/300], Average Loss: 0.4404, ACC: 0.5786766940139769
Epoch [15/300], Average Loss: 0.4295, ACC: 0.5926769978729851
Epoch [16/300], Average Loss: 0.4166, ACC: 0.6025524156791241
Epoch [17/300]

Grand Mean,▁
Grand Mean,0.8597


wandb: Agent Starting Run: 7egcsiz2 with config:
wandb: 	batch_size: 4096
wandb: 	data_augmentation_multiple: 5
wandb: 	dropout: 0.6


...

Grand Mean,▁
Grand Mean,0.84398


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: r4gqyogv with config:
wandb: 	batch_size: 4096
wandb: 	data_augmentation_multiple: 3
wandb: 	dropout: 0.05


Epoch [1/300], Average Loss: 0.7436, ACC: 0.19477317554240473
Epoch [2/300], Average Loss: 0.5488, ACC: 0.4884311940524984
Epoch [3/300], Average Loss: 0.4704, ACC: 0.5467152177211356
Epoch [4/300], Average Loss: 0.4072, ACC: 0.6920042482172651
Epoch [5/300], Average Loss: 0.3678, ACC: 0.741511151570322
Epoch [6/300], Average Loss: 0.3345, ACC: 0.8093991807009548
Epoch [7/300], Average Loss: 0.3095, ACC: 0.8462676376877559
Epoch [8/300], Average Loss: 0.2886, ACC: 0.8826050675163096
Epoch [9/300], Average Loss: 0.2700, ACC: 0.9075254134425729
Epoch [10/300], Average Loss: 0.2549, ACC: 0.9314974965862537
Epoch [11/300], Average Loss: 0.2435, ACC: 0.9409801244120771
Epoch [12/300], Average Loss: 0.2314, ACC: 0.9510696404187529
Epoch [13/300], Average Loss: 0.2204, ACC: 0.9633591260810198
Epoch [14/300], Average Loss: 0.2113, ACC: 0.972614170839023
Epoch [15/300], Average Loss: 0.2031, ACC: 0.9775451373084508
Epoch [16/300], Average Loss: 0.1966, ACC: 0.977545137308451
Epoch [17/300], Ave

wandb: Ctrl + C detected. Stopping sweep.


Epoch [224/300], Average Loss: 0.0080, ACC: 0.9949931725079654


In [17]:
smote_multiple

{1695177740: [0.8596975266933441,
  0.830520266182698,
  0.8576225045372052,
  0.8726104080677033,
  0.8708862662315369,
  0.8668481647968292],
 1695179116: [0.8546360194683075,
  0.8200241984271022,
  0.8523290986085904,
  0.8659255862236023,
  0.8716122210025787,
  0.8632889926433563],
 1695180009: [0.834256112575531,
  0.780096793708409,
  0.8304244807420851,
  0.8441117227077484,
  0.8689453423023223,
  0.8477021753787994],
 1695181341: [0.8635964870452881,
  0.8406836055656383,
  0.8620387174833635,
  0.875030243396759,
  0.8709770023822785,
  0.8692528903484344],
 1695182166: [0.8513208210468293,
  0.8305505142165759,
  0.8498790078644888,
  0.86343013048172,
  0.8562310874462128,
  0.8565134227275848],
 1695183013: [0.8510556697845459,
  0.8202056866303689,
  0.8489816495261142,
  0.8582123339176178,
  0.8691117107868195,
  0.8587669253349304],
 1695184453: [0.8551875412464142,
  0.8114942528735632,
  0.8524752974389997,
  0.8645845890045166,
  0.8810546517372131,
  0.8663289248

In [18]:
import json
file = open("/home/kongge/projects/new_protT5/data/dictionary_data_no_auto_threshold.json", "w")
json.dump(smote_multiple, file)
file.close()